# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [47]:
import graphlab
import numpy
graphlab.product_key.set_product_key('7361-BA94-D081-785E-2308-3F6E-9D04-0EB6')

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [48]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [49]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [50]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [51]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.632799     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.675879     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [52]:
model_all.get("coefficients").print_rows(num_rows = len(all_features)+1)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [53]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [54]:
l1_penalty_set = numpy.logspace(1, 7, num=13)
output_name = 'price'
print l1_penalty_set

[  1.00000000e+01   3.16227766e+01   1.00000000e+02   3.16227766e+02
   1.00000000e+03   3.16227766e+03   1.00000000e+04   3.16227766e+04
   1.00000000e+05   3.16227766e+05   1.00000000e+06   3.16227766e+06
   1.00000000e+07]


In [55]:
def rss(model, data, features_list, output_name):
    predictions = model.predict(data[features_list])
    errors = predictions - data[output_name]
    squared_errors = errors ** 2
    return squared_errors.sum()

In [56]:
def build_l1_penalty_validation_errors_sframe(training_data, validation_data, features_list, output_name, l1_penalty_set):
    l1_penalty_errors_sframe = graphlab.SFrame()
    l1_penalty_errors_sframe['penalty'] = graphlab.SArray(l1_penalty_set)
    validation_errors = []
    for l1_penalty in l1_penalty_set:
        model = graphlab.linear_regression.create(training_data, target=output_name, features=features_list,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty)
        validation_errors.append(rss(model, validation_data, features_list, output_name))
        model.get("coefficients").print_rows(num_rows = len(features_list)+1)
    l1_penalty_errors_sframe['errors'] = graphlab.SArray(validation_errors)
    return l1_penalty_errors_sframe

In [57]:
l1_penalty_errors_sframe = build_l1_penalty_validation_errors_sframe(training, validation, all_features, output_name, l1_penalty_set)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 0.389116     | 5659149.988446     | 285408.885485 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None |   19604.7163508    |  None  |
|     bedrooms     |  None |   7922.62023075    |  None  |
| bedrooms_square  |  None |   928.112147889    |  None  |
|    bathrooms     |  None |    25392.743443    |  None  |
|   sqft_living    |  None |   39.1083048767    |  None  |
| sqft_living_sqrt |  None |   1123.95233925    |  None  |
|     sqft_lot     |  None | -0.000823987151992 |  None  |
|  sqft_lot_sqrt   |  None |   147.859054391    |  None  |
|      floors      |  None |    21154.353574    |  None  |
|  floors_square   |  None |    12871.042877    |  None  |
|    waterfront    |  None |   598538.378522    |  None  |
|       view       |  None |   93089.3048849    |  None  |
|    condition     |  None |   6584.64358167    |  None  |
|      grade       |  None |   6200.37319739    |  None 

In [58]:
l1_penalty_errors_sframe.print_rows(num_rows=len(l1_penalty_set+1))

+---------------+-------------------+
|    penalty    |       errors      |
+---------------+-------------------+
|      10.0     | 6.25766285142e+14 |
| 31.6227766017 | 6.25766285362e+14 |
|     100.0     | 6.25766286058e+14 |
| 316.227766017 | 6.25766288257e+14 |
|     1000.0    | 6.25766295212e+14 |
| 3162.27766017 | 6.25766317206e+14 |
|    10000.0    | 6.25766386761e+14 |
| 31622.7766017 | 6.25766606749e+14 |
|    100000.0   | 6.25767302792e+14 |
| 316227.766017 | 6.25769507644e+14 |
|   1000000.0   | 6.25776517727e+14 |
| 3162277.66017 | 6.25799062845e+14 |
|   10000000.0  | 6.25883719085e+14 |
+---------------+-------------------+
[13 rows x 2 columns]



In [59]:
def find_best_l1_penalty(l1_penalty_errors_sframe):
    min_validation_error = l1_penalty_errors_sframe['errors'].min()
    return l1_penalty_errors_sframe[l1_penalty_errors_sframe['errors'] == min_validation_error][0]['penalty']

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [60]:
best_l1_penalty = find_best_l1_penalty(l1_penalty_errors_sframe)
print best_l1_penalty

10.0


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [61]:
model = graphlab.linear_regression.create(training, target=output_name, features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=best_l1_penalty)
model.get("coefficients").print_rows(num_rows = len(all_features)+1)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 0.439908     | 5655864.782013     | 285354.826809 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [62]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [63]:
l1_penalty_values = numpy.logspace(8, 10, num=20)
print l1_penalty_values

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [64]:
def build_l1_penalty_coefficients_sframe(training_data, features_list, output_name, l1_penalty_set):
    l1_penalty_coefficients_sframe = graphlab.SFrame()
    l1_penalty_coefficients_sframe['penalty'] = graphlab.SArray(l1_penalty_set)
    coefficients_array = []
    nnz_array = []
    for l1_penalty in l1_penalty_set:
        model = graphlab.linear_regression.create(training_data, target=output_name, features=features_list,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty)
        coefficients = model['coefficients']['value']
        coefficients_array.append(coefficients)
        nnz_array.append(coefficients.nnz())
        model.get("coefficients").print_rows(num_rows = len(features_list)+1)
    l1_penalty_coefficients_sframe['coefficients'] = graphlab.SArray(coefficients_array)
    l1_penalty_coefficients_sframe['nnz'] = graphlab.SArray(nnz_array)
    return l1_penalty_coefficients_sframe

In [65]:
l1_penalty_coefficients_sframe = build_l1_penalty_coefficients_sframe(training, all_features, output_name, l1_penalty_values)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000012  | 0.366731     | 7187431.218291     | 380705.223778 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 564482.136844 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None |      0.0      |  None  |
|   sqft_living    |  None |      0.0      |  None  |
| sqft_living_sqrt |  None |      0.0      |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None |      0.0      |  None  |
|    sqft_above    |  None |      0.0      |  None  |
|  sqft_basement   |  None |

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [66]:
l1_penalty_coefficients_sframe.print_rows(num_rows= len(l1_penalty_coefficients_sframe)+1)

+---------------+-------------------------------+-----+
|    penalty    |          coefficients         | nnz |
+---------------+-------------------------------+-----+
|  100000000.0  | [25090.9173672, 7789.17706... |  18 |
|  127427498.57 | [26746.6619366, 7743.97904... |  18 |
| 162377673.919 | [28873.1810166, 7691.04707... |  18 |
| 206913808.111 | [31564.6064733, 7618.56776... |  18 |
| 263665089.873 | [34954.6717432, 7515.37748... |  17 |
| 335981828.628 | [39314.6150486, 7394.88288... |  17 |
| 428133239.872 | [44909.3379183, 7253.29314... |  17 |
| 545559478.117 | [52030.0300806, 7070.21661... |  17 |
| 695192796.178 | [61121.2820244, 6842.36051... |  17 |
|  885866790.41 | [72653.7628357, 6537.49853... |  16 |
| 1128837891.68 | [87298.8276811, 6137.40172... |  15 |
| 1438449888.29 | [105843.800016, 5589.85410... |  15 |
| 1832980710.83 | [129178.714228, 4784.47090... |  13 |
| 2335721469.09 | [158796.200904, 3707.13962... |  12 |
| 2976351441.63 | [196100.937806, 2181.57432... 

In [67]:
l1_penalty_min_sframe = l1_penalty_coefficients_sframe[l1_penalty_coefficients_sframe['nnz'] > max_nonzeros]
l1_penalty_min = l1_penalty_min_sframe['penalty'].max()
print l1_penalty_min

2976351441.63


In [68]:
l1_penalty_max_sframe = l1_penalty_coefficients_sframe[l1_penalty_coefficients_sframe['nnz'] < max_nonzeros]
l1_penalty_max = l1_penalty_max_sframe['penalty'].min()
print l1_penalty_max

3792690190.73


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [69]:
l1_penalty_values = numpy.linspace(l1_penalty_min,l1_penalty_max,20)
print l1_penalty_values

[  2.97635144e+09   3.01931664e+09   3.06228184e+09   3.10524703e+09
   3.14821223e+09   3.19117743e+09   3.23414263e+09   3.27710782e+09
   3.32007302e+09   3.36303822e+09   3.40600341e+09   3.44896861e+09
   3.49193381e+09   3.53489901e+09   3.57786420e+09   3.62082940e+09
   3.66379460e+09   3.70675980e+09   3.74972499e+09   3.79269019e+09]


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [70]:
def build_lasso_model_sframe(training_data, validation_data, features_list, output_name, l1_penalty_set):
    lasso_model_sframe = graphlab.SFrame()
    lasso_model_sframe['penalty'] = graphlab.SArray(l1_penalty_set)
    validation_errors = []
    coefficients_array = []
    nnz_array = []
    for l1_penalty in l1_penalty_set:
        model = graphlab.linear_regression.create(training_data, target=output_name, features=features_list,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty)
        
        model.get("coefficients").print_rows(num_rows = len(features_list)+1)
        validation_errors.append(rss(model, validation_data, features_list, output_name))
        coefficients = model['coefficients']['value']
        coefficients_array.append(coefficients)
        nnz_array.append(coefficients.nnz())
    lasso_model_sframe['errors'] = graphlab.SArray(validation_errors)
    lasso_model_sframe['coefficients'] = graphlab.SArray(coefficients_array)
    lasso_model_sframe['nnz'] = graphlab.SArray(nnz_array)
    return lasso_model_sframe

In [71]:
lasso_model_sframe = build_lasso_model_sframe(training, validation, all_features, output_name, l1_penalty_values)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 0.411231     | 6697293.370032     | 370685.051067 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  240309.75932 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 13840.6399577 |  None  |
|   sqft_living    |  None | 30.5583588298 |  None  |
| sqft_living_sqrt |  None | 592.199469213 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2265.12052556 |  None  |
|    sqft_above    |  None | 27.4878726568 |  None  |
|  sqft_basement   |  None |

In [72]:
lasso_model_sframe.print_rows(num_rows=len(lasso_model_sframe)+1)

+---------------+-------------------+-------------------------------+-----+
|    penalty    |       errors      |          coefficients         | nnz |
+---------------+-------------------+-------------------------------+-----+
| 2976351441.63 | 9.66925692362e+14 | [196100.937806, 2181.57432... |  10 |
| 3019316638.95 | 9.74019450085e+14 | [198563.246218, 2067.01515... |  10 |
| 3062281836.27 | 9.81188367942e+14 | [201025.560405, 1952.47648... |  10 |
| 3105247033.59 | 9.89328342459e+14 | [203443.601, 1825.78864269... |  10 |
| 3148212230.92 | 9.98783211266e+14 | [205805.899501, 1683.39344... |  10 |
| 3191177428.24 | 1.00847716702e+15 | [208163.075463, 1539.55136... |  10 |
| 3234142625.56 | 1.01829878055e+15 | [210520.251425, 1395.70929... |  10 |
| 3277107822.88 | 1.02824799221e+15 | [212877.413342, 1251.86808... |  10 |
|  3320073020.2 | 1.03461690923e+15 | [215235.603644, 1108.36955... |  8  |
| 3363038217.52 | 1.03855473594e+15 | [217598.303815, 966.398500... |  8  |
| 3406003414

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [73]:
print max_nonzeros
result_frame = lasso_model_sframe[lasso_model_sframe['nnz'] == max_nonzeros]
print result_frame

7
+---------------+-------------------+-------------------------------+-----+
|    penalty    |       errors      |          coefficients         | nnz |
+---------------+-------------------+-------------------------------+-----+
| 3448968612.16 | 1.04693748875e+15 | [222253.192544, 661.722717... |  7  |
| 3491933809.48 | 1.05114762561e+15 | [224545.136501, 496.983429... |  7  |
| 3534899006.81 | 1.05599273534e+15 | [226807.047234, 322.098888... |  7  |
| 3577864204.13 | 1.06079953176e+15 | [229077.875448, 149.417508... |  7  |
+---------------+-------------------+-------------------------------+-----+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.


In [74]:
final_best_penalty = find_best_l1_penalty(result_frame)
print final_best_penalty

3448968612.16


In [75]:
result = result_frame[result_frame['penalty'] == final_best_penalty]
coefficients = result['coefficients']
print coefficients

[array('d', [222253.19254432785, 661.7227177822587, 0.0, 15873.957259267981, 32.41022145125964, 690.1147733133256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2899.4202697498786, 30.011575302201045, 0.0, 0.0, 0.0]), ... ]
